<a href="https://colab.research.google.com/github/IamHeeJ/ML_Ex/blob/main/7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#7-2. 심층 신경망

##2개의 층

In [ ]:
# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만듬
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

데이터 준비

In [ ]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)  # 28*28 그림을 1차원배열로 변환

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

(그림p.368)

은닉층에선 시그모이드, 출력층에선 소프트맥스 사용
- 소프트맥스는 확률을 출력하는 함수이므로 중간 밀집층에서 쓸 필요가 x

이 그림이 주어지면 모델을 설계할수 있어야함

**은닉층**


**밀집층**


**은닉층에 활성화 함수를 쓰는 이유?**

(p.369)

활성화 함수 : 밀집층이 하나로 합쳐지지 않도록 방지함

In [ ]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

##심층 신경망 만들기

In [ ]:
model = keras.Sequential([dense1, dense2])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


=> Model: "sequential" <- 모델 이름

층마다 이름, 클래스, 출력 크기, 모델 파라미터 개수 출력

---

Layer(type) : 레이어의 이름, (타입)
- 층을 만들 때 name 매개변수로 이름 지정 가능, 미지정시 자동으로 dense라고 이름이 붙여짐

Output Shape : (샘플 개수, 출력값)
- None : 현재 샘플 개수가 정의되어있지 않음
- (784,)은 한 샘플의 크기(샘플의 수x)

Param : 파라미터 수
- 각각의 노드에 연결된 간선의 수



---

fit()메서드에 훈련 데이터 주입하면 이 데이터를 잘게 나누어 여러번에 걸쳐 경사 하강법 단계 수행

즉, 미니배치 경사 하강법 사용

케라스의 기본 미니배치 크기는 32이며, batch_size 매개변수로 바꿀 수 있음

따라서 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None 설정


## 층을 추가하는 다른 방법

위에서는 Dense 클래스의 객체 dense1, dense2를 만들어 Sequential 클래스에 전달함

=> 이 객체를 따로 저장하여 쓸 일이 없기 때문에  다음처럼 Sequential 클래스의 생성자 안에서 바로 Dense클래스의 객체를 만드는 경우가 많음



In [ ]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


=> 하지만 이 방법은 Sequential 클래스 생성자가 너무 길어지므로 모델의 add()메서드 사용하는 방법이 가장 많이 사용

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


=> 결과는 동일함

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5683 - accuracy: 0.8068
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4115 - accuracy: 0.8501
Epoch 3/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3773 - accuracy: 0.8637
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3529 - accuracy: 0.8728
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3345 - accuracy: 0.8784


=> 추가된 층이 성능을 향상시킴

##렐루 활성화 함수

**렐루(reLU)**
- 입력이 양수일 경우 마치 활성화 함수가 없는것처럼 그냥 입력을 통과시키고, 음수일 경우에는 0으로 만듬
- max(0,z)로 쓸 수도 있음
  - z가 양수면 z 출력, 음수면 0 출력
- 심층 신경망에서 뛰어남
- 이미지출력엔 렐루함수가 좋음

**Flatten**
- 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할
- 가중치나 절편 x
- 입력층 바로 다음에 추가
- 층이라고는 부르지만, 학습하는 층이 아니므로 층 개수로 세지 않음
- 입력값의 차원을 짐작 가능


=> 어떻게 쓰는지, 왜 쓰는지, 등등 알아놓으면 됩니다

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_8 (Dense)             (None, 100)               78500     
                                                                 
 dense_9 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 2ms/step - loss: 0.5371 - accuracy: 0.8098
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3961 - accuracy: 0.8569
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3564 - accuracy: 0.8708
Epoch 4/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3329 - accuracy: 0.8803
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3165 - accuracy: 0.8859


In [ ]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 3ms/step - loss: 0.3860 - accuracy: 0.8645


[0.3860018849372864, 0.8644999861717224]

=> 이전보다 성능 향상

##옵티마이저

**옵티마이저(optimizer)**

- 케라스가 제공하는 다양한 종류의 경사하강법 알고리즘
- 신경망의 가중치와 절편을 학습하기 위한 알고리즘 또는 방법
- 사용할 경사 하강법 알고리즘과 그 파라미터를 지정하는 매개변수

**모멘텀**
- 새로운 가중치와 절편을 계산할 때, 과거의 가중치와 절편의 변화량을 어느정도 반영할 것인지 결정함
- 즉, 한 번에 이동하는 가중치와 절편의 크기에 변화를 줌

**모멘텀 최적화** => 종류는 외울건 없고 그냥 알아보면 됨

(p.382)
- 모멘텀 기본값은 0인데, 이보다 큰 값으로 지정하면 마치 이전의
- 적응적 학습률

- 네스테로프 모멘텀 최적화 : 모멘텀
- adagrad : 그래디언트 제곱을 누적하여 학습률을 나눔
- RMSprop : 디폴트 경사하강법 방법
- Adam : 모멘텀 최적화 + RMSprop

In [ ]:
# SGD 옵티마이저를 사용하려면 compile()메서드의 optimizer 매개변수를 'sgd'로 지정
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
# 위 코드에서 사용한 'sgd' 문자열의 클래스 구현값
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
# SGD클래스의 학습률 값을 0.1로 변경(기본값 0.01)
sgd = keras.optimizers.SGD(learning_rate=0.1)

In [ ]:
# 네스테로프 모멘텀 최적화
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

In [ ]:
# Adagrad 사용
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
# RMSprop 사용
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
# 커스텀 adam 사용
# adam = keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.8, beta_2=0.9)

# 기본값으로 adam 옵티마이저 사용
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5319 - accuracy: 0.8134
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4013 - accuracy: 0.8565
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3590 - accuracy: 0.8697
Epoch 4/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3328 - accuracy: 0.8796
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3117 - accuracy: 0.8847


In [ ]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3488 - accuracy: 0.8714


[0.3488176167011261, 0.8714166879653931]

**adam의 설정값을 직접 지정하여 옵티마이저로 사용하는 방법**

- 변수명이 adam인 객체 생성
  - 설정값을 직접 지정해줌
- compile()에서 optimizer=adam 으로 문자열이 아닌 변수 adam을 사용하기
  - (원래 기본값으로는 optimizer='adam'같은 문자열 형식으로 사용함)


(코드 사진 참고)

★★★시험에 adam 이렇게 설정하라고 하면 코드 쓼줄 알아야됨

In [ ]:
# 강의 추가 코드
# adam의 설정값을 지정하여 옵티마이저 사용
adam = keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.8, beta_2=0.9) #adam의 설정값을 직접 지정

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics='accuracy') #optimizer 매개변수를 변수명 adam으로 설정

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5512 - accuracy: 0.8122
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4079 - accuracy: 0.8547
Epoch 3/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3696 - accuracy: 0.8673
Epoch 4/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3438 - accuracy: 0.8778
Epoch 5/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3243 - accuracy: 0.8826


In [ ]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3539 - accuracy: 0.8756


[0.35390982031822205, 0.8755833506584167]